# Stock Trading Strategy Based on Neural Net Time Series Analysis

For background, please see our previous notebook: lp_notebook.ipynb and nn_regression.ipynb.

The basic idea is to predict the price of a target stock based on historical values (price, volume, etc..) of a set of predictor stocks; an event happens, it ripples through predictors then to the target. In our previous models we found this is difficult... maybe even impossible. Maybe we will have better luck with neural networks (NN).

One possible benefit of NN is they make it easy to combine predictors. Previously we looked for individual stocks as predictors with the thought that we would find a way to combine them later. With the previous models it is even possible we could have two or more predictors that by themselves could not exceed a threshold, but together they would. With NN we would find such a predictors.

## Unbalanced Classes
In the previous analysis, we looked for spikes in the target that lasted 2 days and exceeded a 3% gain. That approach gave us about 10-20 events/year for an average stock. Using a NN classifier on data like that would fail because the algorithm would be right 90% of the time just by predicting "no price increase" everyday. To get around that issue, we tried using a NN regression model to predict the stock price for every day.

### The Regression Model
In a sense the regression model worked really well, which is not all that surprising given that the past prices of the target stock were used for prediction, and the prices of a stock do not change very much day-to-day. However, as a stock buying strategy the results were almost useless because the model was usually a day late.

### Classification Model
Back to a classification NN. One way to balance the classes is to just decrease the price threshold for getting into the positive class. The problem with that approach is it is not consistent with our hypothesis. It is likely that small price increases are not related to any predictor. Adding them as positive samples is just adding noise.

Furthermore, when we were making synthetic data to test the model, we discovered that our model is not designed to handle price increases that happen over multiple days. To accommodate this, we changed how we label the data. A label is positive if:

    1. (price[today] - price[yesterday]) / price[yesterday]  < threshold
    2. (price[tomorrow] - price[today]) / price[today]  >= threshold
    3. today is at least a convolution kernel width after the last positive label

Of course, this does not resolve the unbalanced classes. There are multiple techniques for dealing with unbalanced classes. Throwing away data seems wrong. So whenever possible we prefer to synthesize data to balance the classes. In this case, it is not clear how to do that. So we will solve the problem by under-sampling the majority class.

## Train, Validate and Test
This data is not stationary. So we should intermix the training and validation samples. This is somewhat tricky because each sample extends over a few time points and we do not want the training and validation samples to contain any of the same time points.

In production, we will only be predicting the next day and we will be doing that sequentially. So the test data is not intermixed.

## Needles in a Haystack
There are 5000 stocks in the NASDAQ. The number of possible combinations of predictors for each stock is enormous. The ideal model would allow us to test many predictors at once. This is similar to image classification, where there is an enormous number of pixels. Convolution models work well for this. We will give it a try in our domain.

## 1D Convolution Model
Our hypothesis is that in the few days before the target stock price goes up, there will be a signal in the predictors. Thus each sample is a 2D (conv_window x n_predictors) tensor. Using these samples, each 1D convolution filter has (conv_window x n_predictors) weights. Every predictor has it's own kernel (column) in this filter. 

After the convolution layer, there needs to be at least one dense layer with the number of inputs equal to the number of filters. If we let:


p = number predictors

f = number 1D convolution filters

k = number time points per sample


number of weights = $p*f*k + f \approx p * f * k$

It seems that approach gives too many degrees of freedom, which will exacerbate over-fitting and increase training time. Our intuition is that we do not need a unique kernel for each predictor for each filter. Rather a small number of kernels, maybe a max of 10, would work for any number of predictors.

To accomplish this, we changed the shape of each sample by stacking the conv_window of points for each predictor. Now each 1D convolution filter has only one kernel. To prevent mixing of data between predictors in this stacked configuration, we set the convolution stride to conv_window. The number of outputs from each filter is now equal to the number of predictors. And the dense layer now has f * p inputs. The number of weights is:

number of weights = $k*f + f*p \approx p * f$

We are using k=3, so this modified 1D convolution has a factor of 3 fewer weights.

In [1]:
%matplotlib widget

import matplotlib.pyplot as plt

import nn.nn as nn
import nn.nn2 as nn2
from my_utils.volatility import load_volatility
from nn.window_generators import IntermixedWindowGenerator

plt.rcParams['figure.figsize'] = [8, 5]

config = {
    'start_date': '20170103',
    'end_date': '20181231',
    'price_field': 'Open',
    'predictor_field': 'Open'
}
splits = [0.5, 0.75]
CONV_WIDTH = 3

# Load the most volatile stocks as predictors
results_dir = nn2.get_results_dir(config)

volatility = load_volatility(results_dir, config)
target = 'PLAG'
predictors = [x[0] for x in volatility[0:10]]
dataframe = nn.load_data(target, predictors, config)
n_predictors = dataframe.shape[1]
n_filters = max(min(10, int(n_predictors / 4)), 4)
print('n predictors: {}'.format(n_predictors))
print('n convolution filters: {}'.format(n_filters))
print(predictors)

n predictors: 10
n convolution filters: 4
['CUEN', 'VERB', 'VRME', 'NMTR', 'AEYE', 'PLAG', 'BLNK', 'ANY', 'ELOX', 'CREX']


The next step is to label the data using the method described above:

In [2]:
# For this target we could not get more than 9% of the samples to be in the (+) class
labels, threshold, frac_pos = nn2.make_labels(dataframe.target, CONV_WIDTH, frac_positive=0.09)
print('Percent positive: {}'.format(100 * frac_pos))

Percent positive: 9.362549800796813


The predictor data varies wildly from stock to stock, and over time. Our hypothesis is that we are looking for localized ripples in the predictor. This suggests that the data can be normalized by taking the difference between successive time points and dividing by the value at one of those time points.

In [3]:
norm_df = nn2.diff_norm(dataframe)

Next we need to make the data samples. As mentioned above, we intermixed the train and validation samples. To balance the data, we under-sampled the majority class. 

In [4]:
conv_window = IntermixedWindowGenerator(norm_df, labels, splits, CONV_WIDTH, balanced=True)
model = nn2.limited_filters_conv_model(n_filters, CONV_WIDTH, n_predictors)

It's not obvious what loss function we should use. It is likely that not every price gain in the target is related to a predictor. So we expect a non-zero rate of false negatives, maybe as high as 50%. At the same time, false positives will either tie up our money on buys with meager gains or worse loose money. The way to avoid false positives is to set the threshold high. This will also increase false negatives. But what is the right rate of false negatives? And if we set the threshold too high, we will not make any buys; we might as well skip all this and just put our money in a stock index fund.

For now, we will go with binary-cross-entropy as the loss function.

In [5]:
history = nn.compile_and_fit_classifier(model, conv_window, patience=2, max_epochs=200, verbose=1)

Epoch 1/200
2/2 [==============================] - 2s 714ms/step - loss: 1.0680 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.6232 - prc: 0.5395 - val_loss: 1.1049 - val_tp: 11.0000 - val_fp: 14.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_precision: 0.4400 - val_recall: 1.0000 - val_auc: 0.6948 - val_prc: 0.5690
Epoch 2/200
2/2 [==============================] - 0s 29ms/step - loss: 1.0656 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.5993 - prc: 0.5425 - val_loss: 1.1024 - val_tp: 11.0000 - val_fp: 14.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_precision: 0.4400 - val_recall: 1.0000 - val_auc: 0.7045 - val_prc: 0.5760
Epoch 3/200
2/2 [==============================] - 0s 22ms/step - loss: 1.0633 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.5622 - prc: 0.5005 - val_loss: 1.0999 - val_

2/2 [==============================] - 0s 19ms/step - loss: 1.0169 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.6519 - prc: 0.6369 - val_loss: 1.0499 - val_tp: 11.0000 - val_fp: 14.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_precision: 0.4400 - val_recall: 1.0000 - val_auc: 0.8117 - val_prc: 0.6450
Epoch 23/200
2/2 [==============================] - 0s 20ms/step - loss: 1.0136 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.6699 - prc: 0.6657 - val_loss: 1.0465 - val_tp: 11.0000 - val_fp: 14.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_precision: 0.4400 - val_recall: 1.0000 - val_auc: 0.8052 - val_prc: 0.6559
Epoch 24/200
2/2 [==============================] - 0s 18ms/step - loss: 1.0106 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.6854 - prc: 0.6597 - val_loss: 1.0430 - val_tp: 11.0000

2/2 [==============================] - 0s 16ms/step - loss: 0.9348 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.7165 - prc: 0.7593 - val_loss: 0.9582 - val_tp: 11.0000 - val_fp: 14.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_precision: 0.4400 - val_recall: 1.0000 - val_auc: 0.8636 - val_prc: 0.7090
Epoch 44/200
2/2 [==============================] - 0s 18ms/step - loss: 0.9295 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.7081 - prc: 0.7485 - val_loss: 0.9526 - val_tp: 11.0000 - val_fp: 14.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_precision: 0.4400 - val_recall: 1.0000 - val_auc: 0.8734 - val_prc: 0.7109
Epoch 45/200
2/2 [==============================] - 0s 17ms/step - loss: 0.9253 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.7057 - prc: 0.7534 - val_loss: 0.9469 - val_tp: 11.0000

2/2 [==============================] - 0s 18ms/step - loss: 0.8229 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.7356 - prc: 0.7574 - val_loss: 0.8303 - val_tp: 11.0000 - val_fp: 14.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_precision: 0.4400 - val_recall: 1.0000 - val_auc: 0.8896 - val_prc: 0.7246
Epoch 65/200
2/2 [==============================] - 0s 21ms/step - loss: 0.8167 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.7488 - prc: 0.7802 - val_loss: 0.8233 - val_tp: 11.0000 - val_fp: 14.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_precision: 0.4400 - val_recall: 1.0000 - val_auc: 0.8799 - val_prc: 0.7167
Epoch 66/200
2/2 [==============================] - 0s 18ms/step - loss: 0.8109 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.7297 - prc: 0.7462 - val_loss: 0.8164 - val_tp: 11.0000

2/2 [==============================] - 0s 19ms/step - loss: 0.7117 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.7512 - prc: 0.7624 - val_loss: 0.7045 - val_tp: 11.0000 - val_fp: 14.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_precision: 0.4400 - val_recall: 1.0000 - val_auc: 0.8734 - val_prc: 0.7842
Epoch 86/200
2/2 [==============================] - 0s 19ms/step - loss: 0.7076 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.7500 - prc: 0.7564 - val_loss: 0.7000 - val_tp: 11.0000 - val_fp: 14.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_precision: 0.4400 - val_recall: 1.0000 - val_auc: 0.8766 - val_prc: 0.7873
Epoch 87/200
2/2 [==============================] - 0s 18ms/step - loss: 0.7027 - tp: 19.0000 - fp: 22.0000 - tn: 0.0000e+00 - fn: 0.0000e+00 - precision: 0.4634 - recall: 1.0000 - auc: 0.7656 - prc: 0.7736 - val_loss: 0.6957 - val_tp: 11.0000

Epoch 107/200
2/2 [==============================] - 0s 18ms/step - loss: 0.6631 - tp: 16.0000 - fp: 9.0000 - tn: 13.0000 - fn: 3.0000 - precision: 0.6400 - recall: 0.8421 - auc: 0.7811 - prc: 0.8034 - val_loss: 0.6509 - val_tp: 8.0000 - val_fp: 3.0000 - val_tn: 11.0000 - val_fn: 3.0000 - val_precision: 0.7273 - val_recall: 0.7273 - val_auc: 0.8344 - val_prc: 0.7802
Epoch 108/200
2/2 [==============================] - 0s 17ms/step - loss: 0.6624 - tp: 16.0000 - fp: 9.0000 - tn: 13.0000 - fn: 3.0000 - precision: 0.6400 - recall: 0.8421 - auc: 0.7763 - prc: 0.7905 - val_loss: 0.6502 - val_tp: 8.0000 - val_fp: 3.0000 - val_tn: 11.0000 - val_fn: 3.0000 - val_precision: 0.7273 - val_recall: 0.7273 - val_auc: 0.8442 - val_prc: 0.7863
Epoch 109/200
2/2 [==============================] - 0s 23ms/step - loss: 0.6615 - tp: 16.0000 - fp: 9.0000 - tn: 13.0000 - fn: 3.0000 - precision: 0.6400 - recall: 0.8421 - auc: 0.7811 - prc: 0.7969 - val_loss: 0.6495 - val_tp: 8.0000 - val_fp: 3.0000 - val_tn:

2/2 [==============================] - 0s 18ms/step - loss: 0.6505 - tp: 12.0000 - fp: 3.0000 - tn: 19.0000 - fn: 7.0000 - precision: 0.8000 - recall: 0.6316 - auc: 0.8122 - prc: 0.8311 - val_loss: 0.6399 - val_tp: 7.0000 - val_fp: 2.0000 - val_tn: 12.0000 - val_fn: 4.0000 - val_precision: 0.7778 - val_recall: 0.6364 - val_auc: 0.8506 - val_prc: 0.7974
Epoch 130/200
2/2 [==============================] - 0s 17ms/step - loss: 0.6497 - tp: 12.0000 - fp: 3.0000 - tn: 19.0000 - fn: 7.0000 - precision: 0.8000 - recall: 0.6316 - auc: 0.8038 - prc: 0.8262 - val_loss: 0.6394 - val_tp: 7.0000 - val_fp: 2.0000 - val_tn: 12.0000 - val_fn: 4.0000 - val_precision: 0.7778 - val_recall: 0.6364 - val_auc: 0.8442 - val_prc: 0.7963
Epoch 131/200
2/2 [==============================] - 0s 20ms/step - loss: 0.6491 - tp: 12.0000 - fp: 3.0000 - tn: 19.0000 - fn: 7.0000 - precision: 0.8000 - recall: 0.6316 - auc: 0.8110 - prc: 0.8331 - val_loss: 0.6388 - val_tp: 5.0000 - val_fp: 1.0000 - val_tn: 13.0000 - val

2/2 [==============================] - 0s 18ms/step - loss: 0.6405 - tp: 8.0000 - fp: 0.0000e+00 - tn: 22.0000 - fn: 11.0000 - precision: 1.0000 - recall: 0.4211 - auc: 0.8421 - prc: 0.8611 - val_loss: 0.6332 - val_tp: 3.0000 - val_fp: 1.0000 - val_tn: 13.0000 - val_fn: 8.0000 - val_precision: 0.7500 - val_recall: 0.2727 - val_auc: 0.8539 - val_prc: 0.8133
Epoch 152/200
2/2 [==============================] - 0s 18ms/step - loss: 0.6400 - tp: 8.0000 - fp: 0.0000e+00 - tn: 22.0000 - fn: 11.0000 - precision: 1.0000 - recall: 0.4211 - auc: 0.8445 - prc: 0.8635 - val_loss: 0.6330 - val_tp: 3.0000 - val_fp: 1.0000 - val_tn: 13.0000 - val_fn: 8.0000 - val_precision: 0.7500 - val_recall: 0.2727 - val_auc: 0.8539 - val_prc: 0.8133
Epoch 153/200
2/2 [==============================] - 0s 19ms/step - loss: 0.6394 - tp: 8.0000 - fp: 0.0000e+00 - tn: 22.0000 - fn: 11.0000 - precision: 1.0000 - recall: 0.4211 - auc: 0.8445 - prc: 0.8635 - val_loss: 0.6328 - val_tp: 3.0000 - val_fp: 1.0000 - val_tn: 1

2/2 [==============================] - 0s 18ms/step - loss: 0.6299 - tp: 11.0000 - fp: 0.0000e+00 - tn: 22.0000 - fn: 8.0000 - precision: 1.0000 - recall: 0.5789 - auc: 0.8684 - prc: 0.8825 - val_loss: 0.6295 - val_tp: 4.0000 - val_fp: 1.0000 - val_tn: 13.0000 - val_fn: 7.0000 - val_precision: 0.8000 - val_recall: 0.3636 - val_auc: 0.8474 - val_prc: 0.8023
Epoch 174/200
2/2 [==============================] - 0s 18ms/step - loss: 0.6294 - tp: 11.0000 - fp: 0.0000e+00 - tn: 22.0000 - fn: 8.0000 - precision: 1.0000 - recall: 0.5789 - auc: 0.8684 - prc: 0.8825 - val_loss: 0.6292 - val_tp: 4.0000 - val_fp: 1.0000 - val_tn: 13.0000 - val_fn: 7.0000 - val_precision: 0.8000 - val_recall: 0.3636 - val_auc: 0.8474 - val_prc: 0.8023
Epoch 175/200
2/2 [==============================] - 0s 18ms/step - loss: 0.6289 - tp: 11.0000 - fp: 0.0000e+00 - tn: 22.0000 - fn: 8.0000 - precision: 1.0000 - recall: 0.5789 - auc: 0.8648 - prc: 0.8801 - val_loss: 0.6290 - val_tp: 4.0000 - val_fp: 1.0000 - val_tn: 1

2/2 [==============================] - 0s 28ms/step - loss: 0.6186 - tp: 11.0000 - fp: 1.0000 - tn: 21.0000 - fn: 8.0000 - precision: 0.9167 - recall: 0.5789 - auc: 0.8744 - prc: 0.8876 - val_loss: 0.6255 - val_tp: 4.0000 - val_fp: 1.0000 - val_tn: 13.0000 - val_fn: 7.0000 - val_precision: 0.8000 - val_recall: 0.3636 - val_auc: 0.8604 - val_prc: 0.8040
Epoch 196/200
2/2 [==============================] - 0s 34ms/step - loss: 0.6183 - tp: 11.0000 - fp: 1.0000 - tn: 21.0000 - fn: 8.0000 - precision: 0.9167 - recall: 0.5789 - auc: 0.8780 - prc: 0.8931 - val_loss: 0.6254 - val_tp: 4.0000 - val_fp: 1.0000 - val_tn: 13.0000 - val_fn: 7.0000 - val_precision: 0.8000 - val_recall: 0.3636 - val_auc: 0.8604 - val_prc: 0.8040
Epoch 197/200
2/2 [==============================] - 0s 22ms/step - loss: 0.6177 - tp: 11.0000 - fp: 1.0000 - tn: 21.0000 - fn: 8.0000 - precision: 0.9167 - recall: 0.5789 - auc: 0.8792 - prc: 0.8937 - val_loss: 0.6252 - val_tp: 4.0000 - val_fp: 1.0000 - val_tn: 13.0000 - val

Here is the summary of the model.

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 10, 3)             0         
_________________________________________________________________
reshape (Reshape)            (None, 1, 30, 1)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1, 10, 4)          16        
_________________________________________________________________
flatten (Flatten)            (None, 40)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 41        
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


Here are the results:

In [7]:
auc_train = model.evaluate(x=conv_window.train[0], y=conv_window.train[1], verbose=0)[7]
auc_val = model.evaluate(x=conv_window.val[0], y=conv_window.val[1], verbose=0)[7]
print('Train AUC: {}'.format(auc_train))
print('Val   AUC: {}'.format(auc_val))

Train AUC: 0.8791866302490234
Val   AUC: 0.8603895902633667


The value of the ROC AUC is for training is much higher than validation, indicating over-fitting.

This is confirmed by the plot of the loss functions:

In [8]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.title('Loss')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Loss')

Even with over-fitting, the validation ROC AUC is near enough to 0.5 to indicate that these predictors are not predictors for this target. Which brings us back to the need to efficiently search the space of predictors.

## Efficiently Looking for Predictors
We can search the predictor space faster if we can test lots of predictors at once. How many should that be?

To answer that question, we wrote some code to modify a predictor by adding a kernel to the predictor exactly before each positive class in the labels. When we ran the modified predictor along with the target through our model we got a perfect fit of both the training and validation data.

This continued when we add 9 other predictors that did not predict this target. With 99 others, we got an AUC in the high 0.90s. With 499 others AUC was around 0.8. So we probably could search in blocks of 500.

## Running on Test Data
At this point there is no need to run the model on the test data because we have not found a set of predictors that work on validation.

When we find a set of predictors that work, we will update the model weights every week as we run it on the test data.
